In [1]:
import import_ipynb
import os.path
from SymbolTable import SymbolTable
from VMWriter import VMWriter

importing Jupyter notebook from SymbolTable.ipynb
importing Jupyter notebook from VMWriter.ipynb


In [2]:
class CompilationEngine:
    opObj = {
        '+': 'add',
        '-': 'sub',
        '&amp;': 'and',
        '|': 'or',
        '&lt;': 'lt',
        '&gt;': 'gt',
        '=': 'eq',
        '/': 'call Math.divide 2',
        '*': 'call Math.multiply 2'
    }
    unaryOp = {
        '-':'neg',
        '~':'not',
    }
    keywordConstant = ['true', 'false', 'null', 'this']
    ifTrueLabel = 'IF_TRUE'
    ifFalseLabel = 'IF_FALSE'
    ifEndLabel = 'IF_END'
    whileLabel = 'WHILE_EXP'
    whileEndLabel = 'WHILE_END'
    
    
    def __init__(self, tags_tokens, file):
        self.tags_tokens = tags_tokens
        self.length = len(tags_tokens)
        self.pc = -1
        
        self.Name = ''
        self.Type = ''
        self.Kind = ''
        self.Index = 0
        self.symbelTabel = SymbolTable()
        
        # 程序控制
        self.ifIndex = 0
        self.whileIndex = 0
        
        # 子程序定义
        self.className = ''
        self.subroutineName = ''
        self.subroutineType = ''
        self.returnVoid = True
        
        self.vmWriter = VMWriter(file)
        
        self.complieClass()
        self.vmWriter.close()
        
    def getNextTagToken(self):
        self.pc += 1
        return self.tags_tokens[self.pc]
    
    def lookNextTagToken(self):
        return self.tags_tokens[self.pc+1]
    
    def lookCurrentTagToken(self):
        return self.tags_tokens[self.pc]
        
    def complieClass(self):
        temp = self.getNextTagToken()
        self.key, self.val, self.line = temp[0], temp[1], temp[2]
        if self.val == 'class':
            
            temp = self.getNextTagToken()
            self.key, self.val, self.line = temp[0], temp[1], temp[2]
            if self.key == 'identifier':
                self.className = self.val
                
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                if self.val == '{':
                    
                    while self.pc < self.length:
                        temp = self.getNextTagToken()
                        self.key, self.val, self.line = temp[0], temp[1], temp[2]
                        if self.val == '}':
                            if self.pc != self.length - 1:
                                raise Exception("\n--> " + self.line + ' ' + self.val + " before code")
                            break
                            
                        if self.val in ['static', 'field']:
                            self.compileClassVarDec()
                            if self.pc == self.length - 1:
                                raise Exception("\n--> " + self.line + ' ' + self.val + " need a }")
                        elif self.val in ['constructor', 'function', 'method']:
                            self.compileSubroutineDec()
                            if self.pc == self.length - 1:
                                raise Exception("\n--> " + self.line + ' ' + self.val + " need a }")
                        else:
                            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a static|field|constructor|function|method")
                else:
                    raise Exception("\n--> " + self.line + ' ' + self.val + " should be a {")
            else:
                raise Exception("\n--> " + self.line + ' ' + self.val + " should be a identifier")
        else:
            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a class")
        
    def compileClassVarDec(self):
        self.Kind = self.val
        
        temp = self.getNextTagToken()
        self.key, self.val, self.line = temp[0], temp[1], temp[2]
        if self.val in ['int', 'char', 'boolean'] or self.key == 'identifier':
            self.Type = self.val
            
            temp = self.getNextTagToken()
            self.key, self.val, self.line = temp[0], temp[1], temp[2]
            if self.key == 'identifier':
                self.symbelTabel.Define(self.val, self.Type, self.Kind)
                
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                while self.val != ';':
                    if self.val == ',':
                        
                        temp = self.getNextTagToken()
                        self.key, self.val, self.line = temp[0], temp[1], temp[2]
                        if self.key == 'identifier':
                            self.symbelTabel.Define(self.val, self.Type, self.Kind)
                            
                            temp = self.getNextTagToken()
                            self.key, self.val, self.line = temp[0], temp[1], temp[2]
                        else:
                            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a identifier")
                    else:
                        raise Exception("\n--> " + self.line + ' ' + self.val + " should be a ,")
            else:
                raise Exception("\n--> " + self.line + ' ' + self.val + " should be a identifier")
        else:
            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a type")
        
        return
    
    def compileSubroutineDec(self):
        self.subroutineType = self.val
        self.ifIndex = 0
        self.whileIndex = 0
        self.symbelTabel.startSubroutine()
        
        temp = self.getNextTagToken()
        self.key, self.val, self.line = temp[0], temp[1], temp[2]
        if self.val in ['void', 'int', 'char', 'boolean'] or self.key == 'identifier':
            if self.val == 'void':
                self.returnVoid = True
            
            temp = self.getNextTagToken()
            self.key, self.val, self.line = temp[0], temp[1], temp[2]
            if self.key == 'identifier':
                self.subroutineName = self.val
            
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                if self.val == '(':
            
                    temp = self.getNextTagToken()
                    self.key, self.val, self.line = temp[0], temp[1], temp[2]
                    if self.subroutineType == 'method':
                        self.symbelTabel.argumentIndex += 1
                    self.compileParameterList()
                    
                    self.compileSubroutineBody()
                else:
                    raise Exception("\n--> " + self.line + ' ' + self.val + " should be a (")
            else:
                raise Exception("\n--> " + self.line + ' ' + self.val + " should be a identifier")
        else:
            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a void or type")

        return
    
    def compileParameterList(self):
        self.Kind = 'argument'
        
        if self.val != ')':
            if self.val in ['int', 'char', 'boolean'] or self.key == 'identifier':
                self.Type = self.val
                
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                if self.key == 'identifier':
                    self.symbelTabel.Define(self.val, self.Type, self.Kind)
            
                    temp = self.getNextTagToken()
                    self.key, self.val, self.line = temp[0], temp[1], temp[2]
                    while self.val != ')':
                        if self.val == ',':

                            temp = self.getNextTagToken()
                            self.key, self.val, self.line = temp[0], temp[1], temp[2]
                            if self.val in ['int', 'char', 'boolean'] or self.key == 'identifier':
                                self.Type = self.val

                                temp = self.getNextTagToken()
                                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                                if self.key == 'identifier':
                                    self.symbelTabel.Define(self.val, self.Type, self.Kind)

                                    temp = self.getNextTagToken()
                                    self.key, self.val, self.line = temp[0], temp[1], temp[2]
                                else:
                                    raise Exception("\n--> " + self.line + ' ' + self.val + " should be a identifier")
                            else:
                                raise Exception("\n--> " + self.line + ' ' + self.val + " should be a type")
                        else:
                            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a ,")
                else:
                    raise Exception("\n--> " + self.line + ' ' + self.val + " should be a identifier")
            else:
                raise Exception("\n--> " + self.line + ' ' + self.val + " should be a ) or type")
        
    def compileSubroutineBody(self):
        
        temp = self.getNextTagToken()
        self.key, self.val, self.line = temp[0], temp[1], temp[2]
        if self.val == '{':

            temp = self.getNextTagToken()
            self.key, self.val, self.line = temp[0], temp[1], temp[2]
            while self.val == 'var':
                self.compileVarDec()

                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                
            self.vmWriter.writeFunction(self.className + '.' + self.subroutineName, self.symbelTabel.VarCount('var'))
            
            if self.subroutineType == 'constructor':
                fieldVarCount = self.symbelTabel.VarCount('field')
                self.vmWriter.writePush('constant', fieldVarCount) # 为成员变量分配空间
                self.vmWriter.writeCall('Memory.alloc', 1)
                self.vmWriter.writePop('pointer', 0) # 将分配空间的得到的指针放到this里
            if self.subroutineType == 'method':
                self.vmWriter.writePush('argument', 0)
                self.vmWriter.writePop('pointer', 0) # 将参数传递进来的指针放到this里
                
            self.compileStatements()
            if self.val != '}':
                raise Exception("\n--> " + self.line + ' ' + self.val + " should be a var|let|if|while|do|return or }")
        else:
            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a {")
        
    def compileVarDec(self):
        self.Kind = 'var'
        
        temp = self.getNextTagToken()
        self.key, self.val, self.line = temp[0], temp[1], temp[2]
        if self.val in ['int', 'char', 'boolean'] or self.key == 'identifier':
            self.Type = self.val
            
            temp = self.getNextTagToken()
            self.key, self.val, self.line = temp[0], temp[1], temp[2]
            if self.key == 'identifier':
                self.symbelTabel.Define(self.val, self.Type, self.Kind)
                
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                while self.val != ';':
                    if self.val == ',':
                
                        temp = self.getNextTagToken()
                        self.key, self.val, self.line = temp[0], temp[1], temp[2]
                        if self.key == 'identifier':
                            self.symbelTabel.Define(self.val, self.Type, self.Kind)

                            temp = self.getNextTagToken()
                            self.key, self.val, self.line = temp[0], temp[1], temp[2]
                        else:
                            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a identifier")
                    else:
                        raise Exception("\n--> " + self.line + ' ' + self.val + " should be a , or ;")
            else:
                raise Exception("\n--> " + self.line + ' ' + self.val + " should be a identifier")
        else:
            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a type")
    
    def compileStatements(self):
        
        while self.val in ['let', 'if', 'while', 'do', 'return']:
            if self.val == 'let':
                self.compileLet()
            elif self.val == 'if':
                self.compileIf()
            elif self.val == 'while':
                self.compileWhile()
            elif self.val == 'do':
                self.compileDo()
            elif self.val == 'return':
                self.compileReturn()
    
    def compileLet(self):
        
        temp = self.getNextTagToken()
        self.key, self.val, self.line = temp[0], temp[1], temp[2]
        if self.key == 'identifier':
            isArray = False
            Kind = self.symbelTabel.KindOf(self.val)
            Index = self.symbelTabel.IndexOf(self.val)
            
            temp = self.getNextTagToken()
            self.key, self.val, self.line = temp[0], temp[1], temp[2]
            if self.val == '[':
                isArray = True
                
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                self.compileExpression()
                
                self.vmWriter.writePush(Kind, Index)
                self.vmWriter.writeArithmetic('add')
                if self.val != ']':
                    raise Exception("\n--> " + self.line + ' ' + self.val + " should be a ]")
                
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
            if self.val == '=':
                
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                self.compileExpression()
                
                if isArray == True: # 如果是数组
                    self.vmWriter.writePop('temp', 0) # 把=右侧结果放入临时区
                    self.vmWriter.writePop('pointer', 1) # 把=左侧指针放入that
                    self.vmWriter.writePush('temp', 0) # 把=右侧结果放回堆栈
                    self.vmWriter.writePop('that', 0) # a[i] = 右侧结果
                else:
                    self.vmWriter.writePop(Kind, Index) # 将返回值弹出到指定的变量位置
                
                if self.val == ';':
                    temp = self.getNextTagToken()
                    self.key, self.val, self.line = temp[0], temp[1], temp[2]
                else:
                    raise Exception("\n--> " + self.line + ' ' + self.val + " should be a ;")
            else:
                raise Exception("\n--> " + self.line + ' ' + self.val + " should be a =")
        else:
            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a identifer")
        return
    
    def compileIf(self):
        
        temp = self.getNextTagToken()
        self.key, self.val, self.line = temp[0], temp[1], temp[2]
        if self.val == '(':
            
            temp = self.getNextTagToken()
            self.key, self.val, self.line = temp[0], temp[1], temp[2]
            self.compileExpression()
            if self.val == ')':
                ifIndex = self.ifIndex
                self.ifIndex += 1
                self.vmWriter.writeIf(self.ifTrueLabel + str(ifIndex))
                self.vmWriter.writeGoto(self.ifFalseLabel + str(ifIndex))
                
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                if self.val == '{':
                    self.vmWriter.writeLabel(self.ifTrueLabel + str(ifIndex))

                    temp = self.getNextTagToken()
                    self.key, self.val, self.line = temp[0], temp[1], temp[2]
                    self.compileStatements()
                    
                    if self.val == '}':

                        temp = self.getNextTagToken()
                        self.key, self.val, self.line = temp[0], temp[1], temp[2]
                        if self.val == 'else':

                            temp = self.getNextTagToken()
                            self.key, self.val, self.line = temp[0], temp[1], temp[2]
                            if self.val == '{':
                                self.vmWriter.writeGoto(self.ifEndLabel + str(ifIndex))
                                self.vmWriter.writeLabel(self.ifFalseLabel + str(ifIndex))

                                temp = self.getNextTagToken()
                                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                                self.compileStatements()
                                
                                self.vmWriter.writeLabel(self.ifEndLabel + str(ifIndex))
                                if self.val == '}':
                                    
                                    temp = self.getNextTagToken()
                                    self.key, self.val, self.line = temp[0], temp[1], temp[2]
                                else:
                                    raise Exception("\n--> " + self.line + ' ' + self.val + " should be a }")
                            else:
                                raise Exception("\n--> " + self.line + ' ' + self.val + " should be a {")
                        else:
                            self.vmWriter.writeLabel(self.ifFalseLabel + str(ifIndex))
                    else:
                        raise Exception("\n--> " + self.line + ' ' + self.val + " should be a }")
                else:
                    raise Exception("\n--> " + self.line + ' ' + self.val + " should be a {")
            else:
                raise Exception("\n--> " + self.line + ' ' + self.val + " should be a )")
        else:
            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a (")
            
        return
    
    def compileWhile(self):
        self.vmWriter.writeLabel(self.whileLabel + str(self.whileIndex))
        
        temp = self.getNextTagToken()
        self.key, self.val, self.line = temp[0], temp[1], temp[2]
        if self.val == '(':
            
            temp = self.getNextTagToken()
            self.key, self.val, self.line = temp[0], temp[1], temp[2]
            self.compileExpression()
            if self.val == ')':
                whileIndex = self.whileIndex
                self.whileIndex += 1
                self.vmWriter.writeArithmetic('not')
                self.vmWriter.writeIf(self.whileEndLabel + str(whileIndex))
                
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                if self.val == '{':

                    temp = self.getNextTagToken()
                    self.key, self.val, self.line = temp[0], temp[1], temp[2]
                    self.compileStatements()
                    self.vmWriter.writeGoto(self.whileLabel + str(whileIndex))
                    
                    if self.val == '}':
                        self.vmWriter.writeLabel(self.whileEndLabel + str(whileIndex))

                        temp = self.getNextTagToken()
                        self.key, self.val, self.line = temp[0], temp[1], temp[2]
                    else:
                        raise Exception("\n--> " + self.line + ' ' + self.val + " should be a }")
                else:
                    raise Exception("\n--> " + self.line + ' ' + self.val + " should be a {")
            else:
                raise Exception("\n--> " + self.line + ' ' + self.val + " should be a )")
        else:
            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a (")
    
    def compileDo(self):
        temp = self.getNextTagToken()
        self.key, self.val, self.line = temp[0], temp[1], temp[2]
        
        self.compileSubroutineCall()
        self.vmWriter.writePop('temp', 0)
        
        if self.val == ';':
            temp = self.getNextTagToken()
            self.key, self.val, self.line = temp[0], temp[1], temp[2]
        else:
            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a ;")

        return
    
    def compileReturn(self):
        
        temp = self.getNextTagToken()
        self.key, self.val, self.line = temp[0], temp[1], temp[2]
        if self.val != ';':
            self.compileExpression()
        elif self.val == ';':
            self.vmWriter.writePush('constant', '0')
        else:
            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a expression or ;")
        
        self.vmWriter.writeReturn()
        
        temp = self.getNextTagToken()
        self.key, self.val, self.line = temp[0], temp[1], temp[2]
    
    def compileExpression(self):
        
        self.compileTerm()
        while self.val in self.opObj:
            op = self.opObj[self.val]
        
            temp = self.getNextTagToken()
            self.key, self.val, self.line = temp[0], temp[1], temp[2]
            self.compileTerm()
            
            self.vmWriter.writeArithmetic(op)
        return
    
    def compileTerm(self):
        if self.key in ['integerConstant', 'stringConstant'] or self.val in self.keywordConstant: # 常量
            if self.key == 'integerConstant': # 常数
                self.vmWriter.writePush('constant', self.val)

                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
            
            if self.key == 'stringConstant': # 字符串
                self.vmWriter.writePush('constant', len(self.val))
                self.vmWriter.writeCall('String.new', 1)
                for char in self.val:
                    self.vmWriter.writePush('constant', ord(char))
                    self.vmWriter.writeCall('String.appendChar', 2)

                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]

            if self.val in self.keywordConstant: # 关键字常量
                if self.val == 'true':
                    self.vmWriter.writePush('constant', '0')
                    self.vmWriter.writeArithmetic('not')
                if self.val == 'false':
                    self.vmWriter.writePush('constant', '0') # 16位全零为假
                if self.val == 'this':
                    self.vmWriter.writePush('pointer', '0')
                if self.val == 'null':
                    self.vmWriter.writePush('constant', '0')

                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
            
        elif self.key == 'identifier': # 变量/数组/子程序调用
            subroutineCallName = self.val
            varName = self.val
        
            temp = self.lookNextTagToken() # 提前看下一个字元
            self.key, self.val, self.line = temp[0], temp[1], temp[2]
            if self.val == '[': # 数组
                
                self.getNextTagToken()
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                self.compileExpression()
                
                Kind = self.symbelTabel.KindOf(varName)
                Index = self.symbelTabel.IndexOf(varName)
                self.vmWriter.writePush(Kind, Index)
                self.vmWriter.writeArithmetic('add')
                self.vmWriter.writePop('pointer', 1) # that = a + i
                self.vmWriter.writePush('that', 0) # a[i]
                
                if self.val == ']':
        
                    temp = self.getNextTagToken()
                    self.key, self.val, self.line = temp[0], temp[1], temp[2]
                else:
                    raise Exception("\n--> " + self.line + ' ' + self.val + " should be a ]")
            elif self.val == '(' or self.val == '.': # 子程序调用
                self.compileSubroutineCall()
            else: # 变量
                Kind = self.symbelTabel.KindOf(varName)
                Index = self.symbelTabel.IndexOf(varName)
                self.vmWriter.writePush(Kind, Index)
                
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                
        elif self.val == '(': # ( expression )
                
            temp = self.getNextTagToken()
            self.key, self.val, self.line = temp[0], temp[1], temp[2]
            self.compileExpression()
            
            if self.val == ')':

                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
            else:
                
                raise Exception("\n--> " + self.line + ' ' + self.val + " should be a )")
        elif self.val in self.unaryOp: # unaryOp term
            op = self.unaryOp[self.val]
                
            temp = self.getNextTagToken()
            self.key, self.val, self.line = temp[0], temp[1], temp[2]
            self.compileTerm()
            
            self.vmWriter.writeArithmetic(op)
        else:
            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a term")
        
        return
    
    def compileExpressionList(self):
        parameterCount = 0
        
        if self.val != ')':
            parameterCount += 1
            self.compileExpression()
            while self.val == ',':
                parameterCount += 1
                
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                self.compileExpression()
        
        return parameterCount
    
    def compileSubroutineCall(self):
        temp = self.lookCurrentTagToken()
        self.key, self.val, self.line = temp[0], temp[1], temp[2]
        
        parameterCount = 0
        
        if self.key == 'identifier':
            subroutineCallName = self.val # 可能是程序也可能是变量/类名
        
            temp = self.getNextTagToken()
            self.key, self.val, self.line = temp[0], temp[1], temp[2]
            if self.val == '(':
                self.vmWriter.writePush('pointer', 0) # 把当前对象指针作为本类的函数的第一个参数
                parameterCount += 1
                subroutineCallName = self.className + '.' + subroutineCallName
                
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                parameterCount += self.compileExpressionList()
                
                self.vmWriter.writeCall(subroutineCallName, parameterCount)
                
                if self.val == ')':
        
                    temp = self.getNextTagToken()
                    self.key, self.val, self.line = temp[0], temp[1], temp[2]
                else:
                    
                    raise Exception("\n--> " + self.line + ' ' + self.val + " should be a )")
            elif self.val == '.':
                if self.symbelTabel.TypeOf(subroutineCallName) != 'None':
                    Kind = self.symbelTabel.KindOf(subroutineCallName)
                    Type = self.symbelTabel.TypeOf(subroutineCallName)
                    Index = self.symbelTabel.IndexOf(subroutineCallName)
                    subroutineCallName = Type # 把对象名转换为类名
                    self.vmWriter.writePush(Kind, Index) # 把对象指针作为该对象类的函数第一个参数
                    parameterCount += 1
                
                temp = self.getNextTagToken()
                self.key, self.val, self.line = temp[0], temp[1], temp[2]
                if self.key == 'identifier':
                    subroutineCallName += '.' + self.val # 调用该对象类的函数

                    temp = self.getNextTagToken()
                    self.key, self.val, self.line = temp[0], temp[1], temp[2]
                    if self.val == '(':

                        temp = self.getNextTagToken()
                        self.key, self.val, self.line = temp[0], temp[1], temp[2]
                        parameterCount += self.compileExpressionList()
                        
                        self.vmWriter.writeCall(subroutineCallName, parameterCount)
                        
                        if self.val == ')':

                            temp = self.getNextTagToken()
                            self.key, self.val, self.line = temp[0], temp[1], temp[2]
                        else:
                            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a )")
                    else:
                        raise Exception("\n--> " + self.line + ' ' + self.val + " should be a (")
                else:
                    raise Exception("\n--> " + self.line + ' ' + self.val + " should be a identifer")
            else:
                raise Exception("\n--> " + self.line + ' ' + self.val + " should be a ( or .")
        else:
            raise Exception("\n--> " + self.line + ' ' + self.val + " should be a identifer")